In [1]:
import pandas as pd
import numpy as np

In [2]:
test_df = pd.read_csv('Data/test_clear.csv')
train_df = pd.read_csv('Data/train_clear.csv')

### LightGBM

In [3]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop(['PassengerId'], axis=1).copy()

In [4]:
import lightgbm as lgbm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

parameters = {'n_estimators': [5, 50, 100],
              'learning_rate': np.linspace(0.02,0.16,4),
              'num_leaves': [31, 61],
              'min_data_in_leaf': [20, 30, 40],
              'max_depth': range(3,8),
}

lgbm_model = GridSearchCV(lgbm.LGBMClassifier(), parameters, cv=5).fit(X_train, Y_train)

print(accuracy_score(Y_train, lgbm_model.predict(X_train)))
print(lgbm_model.best_score_)
print(lgbm_model.best_params_)
print(lgbm_model.best_estimator_)

Y_pred = lgbm_model.predict(X_test)
acc_log = round(lgbm_model.score(X_train, Y_train) * 100, 2)
print('ended', acc_log)

0.8395061728395061
0.828278199736363
{'learning_rate': 0.06666666666666667, 'max_depth': 6, 'min_data_in_leaf': 40, 'n_estimators': 50, 'num_leaves': 31}
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.06666666666666667,
               max_depth=6, min_child_samples=20, min_child_weight=0.001,
               min_data_in_leaf=40, min_split_gain=0.0, n_estimators=50,
               n_jobs=-1, num_leaves=31, objective=None, random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)
ended 83.95


In [5]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('titanic-solution-LightGBM.csv', index=False)